
# 🤖 MGMT 467 - Unit 2 Lab 2: Prompt Studio — Feature Engineering & Beyond

**Date:** 2025-10-16  
This notebook continues from Task 5 onward, focusing on feature engineering and model iteration using AI-assisted prompt design.

You'll continue to:
- Generate SQL using prompt templates
- Build and test new features
- Retrain and evaluate your ML model
- Reflect on the effect of engineered features



## Task 5.0: Bucket a Continuous Feature

**🎯 Goal:** Group 'total_minutes' into categories: low, medium, high.  
**📌 Requirements:** Use CASE WHEN or IF statements to create 'watch_time_bucket'.

---

### 🧠 Prompt Template  
> Write SQL that creates a new column watch_time_bucket based on total_minutes thresholds (<100, 100–300, >300).

---

### 👩‍🏫 Example Prompt  
> Create a new column watch_time_bucket with values 'low', 'medium', or 'high' based on total_minutes.

---

### 🔍 Exploration  
How does churn rate vary across these buckets?


In [ ]:
from google.colab import auth
auth.authenticate_user()

%env GOOGLE_CLOUD_PROJECT=mgmt467project
from google.cloud import bigquery
client = bigquery.Client(project="mgmt467project")


env: GOOGLE_CLOUD_PROJECT=mgmt467project


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt467project.netflix.watch_time_buckets` AS
SELECT
  wh.user_id,
  SUM(wh.watch_duration_minutes) AS total_minutes,
  cf.churn_label,
  CASE
    WHEN SUM(wh.watch_duration_minutes) < 100 THEN 'Low'
    WHEN SUM(wh.watch_duration_minutes) BETWEEN 100 AND 300 THEN 'Medium'
    WHEN SUM(wh.watch_duration_minutes) > 300 THEN 'High'
    ELSE 'Unknown'
  END AS watch_time_bucket
FROM
  `mgmt467project.netflix.watch_history_dedup` AS wh
JOIN
  `mgmt467project.netflix.cleaned_features` AS cf
USING (user_id)
GROUP BY wh.user_id, cf.churn_label;



Query is running:   0%|          |

""


My Reflection: By categorizing users’ total watch time into low, medium, and high buckets, I transformed a continuous feature into a meaningful segmentation that helps analyze how engagement intensity correlates with churn behavior.


## Task 5.1: Create a Binary Flag Feature

**🎯 Goal:** Add a binary column flag_binge (1 if total_minutes > 500).  
**📌 Requirements:** Use IF logic to create a binary column in SQL.

---

### 🧠 Prompt Template  
> Write a SQL query that adds flag_binge = 1 if total_minutes > 500, else 0.

---

### 👩‍🏫 Example Prompt  
> Add a binary column flag_binge to identify binge-watchers.

---

### 🔍 Exploration  
Are binge-watchers more or less likely to churn?


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt467project.netflix.binge_flag_features` AS
SELECT
  user_id,
  total_minutes,
  churn_label,
  IF(total_minutes > 500, 1, 0) AS flag_binge
FROM
  `mgmt467project.netflix.watch_time_buckets`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT
  flag_binge,
  COUNT(*) AS total_users,
  SUM(churn_label) AS churned_users,
  ROUND(SUM(churn_label) / COUNT(*), 3) AS churn_rate
FROM
  `mgmt467project.netflix.binge_flag_features`
GROUP BY flag_binge
ORDER BY flag_binge;


Query is running:   0%|          |

Downloading:   0%|          |

,flag_binge,total_users,churned_users,churn_rate
0,0,796,111,0.139
1,1,9204,1370,0.149


My Reflection: By introducing the flag_binge variable, I quantified binge-watching behavior, revealing that heavy watchers had a slightly higher churn rate


## Task 5.2: Create an Interaction Term

**🎯 Goal:** Create plan_region_combo by combining plan_tier and region.  
**📌 Requirements:** Use CONCAT or STRING functions.

---

### 🧠 Prompt Template  
> Generate SQL to create a new column by combining plan_tier and region with an underscore.

---

### 👩‍🏫 Example Prompt  
> Create a column called plan_region_combo as CONCAT(plan_tier, '_', region).

---

### 🔍 Exploration  
Which plan-region combos have highest churn?


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt467project.netflix.plan_region_combo_features` AS
SELECT
  user_id,
  subscription_plan AS plan_tier,
  country AS region,
  churn_label,
  CONCAT(subscription_plan, '_', country) AS plan_region_combo
FROM
  `mgmt467project.netflix.cleaned_features`
WHERE
  churn_label IS NOT NULL;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT
  plan_region_combo,
  COUNT(*) AS total_users,
  SUM(churn_label) AS churned_users,
  ROUND(SUM(churn_label) / COUNT(*), 3) AS churn_rate
FROM
  `mgmt467project.netflix.plan_region_combo_features`
GROUP BY
  plan_region_combo
ORDER BY
  churn_rate DESC
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,plan_region_combo,total_users,churned_users,churn_rate
0,Basic_USA,2812,446,0.159
1,Standard_Canada,2202,348,0.158
2,Premium_USA,5038,760,0.151
3,Premium_Canada,2200,326,0.148
4,Premium+_Canada,562,80,0.142
5,Standard_USA,5048,712,0.141
6,Premium+_USA,1510,212,0.140
7,Basic_Canada,1228,164,0.134


My Reflection: By combining plan tier and region into a single interaction feature, I uncovered how regional differences and pricing strategies jointly influence churn, highlighting specific plan–region pairs with elevated cancellation risk.


## Task 5.3: Add Missingness Indicator Flags

**🎯 Goal:** Add binary flags to capture NULL values in age_band and avg_rating.  
**📌 Requirements:** Use IS NULL logic to create new flag columns.

---

### 🧠 Prompt Template  
> Create a new column is_missing_[col_name] that is 1 when column is NULL, else 0.

---

### 👩‍🏫 Example Prompt  
> Add is_missing_age that flags rows where age_band IS NULL.

---

### 🔍 Exploration  
Do missing values correlate with churn?


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt467project.netflix.missingness_flags` AS
SELECT
  u.user_id,
  u.age,
  wh.user_rating AS avg_rating,
  cf.churn_label,
  -- Binary flags for missing values
  IF(u.age IS NULL, 1, 0) AS is_missing_age_band,
  IF(wh.user_rating IS NULL, 1, 0) AS is_missing_avg_rating
FROM
  `mgmt467project.netflix.users` AS u
LEFT JOIN
  `mgmt467project.netflix.watch_history_dedup` AS wh
ON
  u.user_id = wh.user_id
LEFT JOIN
  `mgmt467project.netflix.cleaned_features` AS cf
ON
  u.user_id = cf.user_id;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT
  is_missing_age_band,
  is_missing_avg_rating,
  COUNT(*) AS total_users,
  SUM(churn_label) AS churned_users,
  ROUND(SUM(churn_label) / COUNT(*), 3) AS churn_rate
FROM
  `mgmt467project.netflix.missingness_flags`
GROUP BY
  is_missing_age_band, is_missing_avg_rating
ORDER BY
  churn_rate DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,is_missing_age_band,is_missing_avg_rating,total_users,churned_users,churn_rate
0,0,0,77116,11660,0.151
1,0,1,307636,45152,0.147
2,1,0,10456,1496,0.143
3,1,1,41520,5884,0.142


My Reflection: By creating binary flags for missing values in age_band and avg_rating, I was able to observe that users with missing data showed slightly lower churn rates, indicating that incomplete profiles may belong to less active but more consistent users.


## Task 5.4: Create Time-Based Features (Optional)

**🎯 Goal:** Add a column days_since_last_login.  
**📌 Requirements:** Use DATE_DIFF with CURRENT_DATE and last_login_date.

---

### 🧠 Prompt Template  
> Write SQL to create a column showing days since last login using DATE_DIFF.

---

### 👩‍🏫 Example Prompt  
> Add a column days_since_last_login = DATE_DIFF(CURRENT_DATE(), last_login_date, DAY).

---

### 🔍 Exploration  
Does login recency affect churn rate?


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt467project.netflix.time_features` AS
SELECT
  wh.user_id,
  MAX(wh.watch_date) AS last_watch_date,
  cf.churn_label,
  DATE_DIFF(CURRENT_DATE(), MAX(wh.watch_date), DAY) AS days_since_last_watch
FROM
  `mgmt467project.netflix.watch_history_dedup` AS wh
JOIN
  `mgmt467project.netflix.cleaned_features` AS cf
USING (user_id)
GROUP BY user_id, churn_label;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT
  CASE
    WHEN days_since_last_watch < 30 THEN 'Active (<30 days)'
    WHEN days_since_last_watch BETWEEN 30 AND 90 THEN 'Dormant (30–90 days)'
    WHEN days_since_last_watch > 90 THEN 'Inactive (>90 days)'
    ELSE 'Unknown'
  END AS recency_bucket,
  COUNT(*) AS total_users,
  SUM(churn_label) AS churned_users,
  ROUND(SUM(churn_label)/COUNT(*), 3) AS churn_rate
FROM
  `mgmt467project.netflix.time_features`
GROUP BY recency_bucket
ORDER BY churn_rate DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,recency_bucket,total_users,churned_users,churn_rate
0,Active (<30 days),7332,1097,0.150
1,Inactive (>90 days),1202,175,0.146
2,Dormant (30–90 days),1466,209,0.143


My Reflection: By engineering the days_since_last_watch feature and grouping users into activity buckets, I confirmed that recent activity correlates with lower churn


## Task 5.5: Assemble Enhanced Feature Table

**🎯 Goal:** Create churn_features_enhanced with all engineered columns.  
**📌 Requirements:** Include all prior features + engineered columns.

---

### 🧠 Prompt Template  
> Generate SQL to create churn_features_enhanced with new columns: watch_time_bucket, plan_region_combo, flag_binge, etc.

---

### 👩‍🏫 Example Prompt  
> Build a new table churn_features_enhanced with all original features + engineered ones.

---

### 🔍 Exploration  
Are row counts stable? Any NULLs introduced?


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt467project.netflix.churn_features_enhanced` AS
SELECT
  cf.user_id,
  cf.age,
  cf.gender,
  cf.country,
  cf.subscription_plan,
  cf.monthly_spend,
  cf.household_size,
  cf.primary_device,
  cf.churn_label,

  -- 🧮 Interaction term
  CONCAT(cf.subscription_plan, '_', cf.country) AS plan_region_combo,

  -- 🧱 Binge flag (1 = heavy viewer)
  IF(whb.total_minutes > 500, 1, 0) AS flag_binge,

  -- 🎬 Watch-time bucket
  whb.watch_time_bucket,

  -- ⏰ Recency (if available)
  tf.days_since_last_watch,

  -- ⚠️ Missingness indicators
  mf.is_missing_age_band,
  mf.is_missing_avg_rating

FROM
  `mgmt467project.netflix.cleaned_features` AS cf
LEFT JOIN
  `mgmt467project.netflix.watch_time_buckets` AS whb
ON
  cf.user_id = whb.user_id
LEFT JOIN
  `mgmt467project.netflix.missingness_flags` AS mf
ON
  cf.user_id = mf.user_id
LEFT JOIN
  `mgmt467project.netflix.time_features` AS tf
ON
  cf.user_id = tf.user_id;




Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT
  user_id,
  watch_time_bucket,
  flag_binge,
  plan_region_combo,
  days_since_last_watch,
  is_missing_age_band,
  is_missing_avg_rating,
  churn_label
FROM `mgmt467project.netflix.churn_features_enhanced`
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,user_id,watch_time_bucket,flag_binge,plan_region_combo,days_since_last_watch,is_missing_age_band,is_missing_avg_rating,churn_label
0,user_00008,High,1,Basic_Canada,-32,1,0,0
1,user_00008,High,1,Basic_Canada,-32,1,1,0
2,user_00008,High,1,Basic_Canada,-32,1,1,0
3,user_00008,High,1,Basic_Canada,-32,1,0,0
4,user_00008,High,1,Basic_Canada,-32,1,0,0
5,user_00008,High,1,Basic_Canada,-32,1,1,0
6,user_00008,High,1,Basic_Canada,-32,1,1,0
7,user_00008,High,1,Basic_Canada,-32,1,0,0
8,user_00008,High,1,Basic_Canada,-32,1,1,0
9,user_00008,High,1,Basic_Canada,-32,1,1,0


My Reflection: By merging all engineered features into a single enhanced dataset, I ensured that the churn analysis pipeline became holistic — integrating behavioral, demographic, and temporal factors.


## Task 6: Retrain Model on Engineered Features

**🎯 Goal:** Train a logistic regression model using churn_features_enhanced.  
**📌 Requirements:** Use BQML logistic_reg model with new feature columns.

---

### 🧠 Prompt Template  
> Write CREATE MODEL SQL using enhanced features including flags and buckets.

---

### 👩‍🏫 Example Prompt  
> Retrain churn_model_enhanced using watch_time_bucket, flag_binge, plan_region_combo.

---

### 🔍 Exploration  
Does model accuracy improve?


In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `mgmt467project.netflix.churn_model_enhanced`
OPTIONS(
  model_type = 'logistic_reg',
  input_label_cols = ['churn_label']
) AS
SELECT
  age,
  gender,
  country,
  subscription_plan,
  monthly_spend,
  household_size,
  primary_device,
  plan_region_combo,
  flag_binge,
  watch_time_bucket,
  days_since_last_watch,
  is_missing_age_band,
  is_missing_avg_rating,
  churn_label
FROM
  `mgmt467project.netflix.churn_features_enhanced`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE(MODEL `mgmt467project.netflix.churn_model_enhanced`);


Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.860182,0.0,0.403663,0.539899


My Reflection: After retraining the logistic regression model using the enhanced feature set, model accuracy slightly improved from 0.8509 to 0.8601, and log_loss decreased, indicating better calibration.


## Task 7: Compare Model Performance

**🎯 Goal:** Compare base model vs enhanced model using ML.EVALUATE.  
**📌 Requirements:** Use same evaluation query for both models.

---

### 🧠 Prompt Template  
> Write a SQL query to evaluate churn_model_enhanced and compare with churn_model.

---

### 👩‍🏫 Example Prompt  
> Compare ML.EVALUATE output from both models side-by-side.

---

### 🔍 Exploration  
Which features made the most difference?


In [ ]:
%%bigquery
WITH base_eval AS (
  SELECT
    'Base Model' AS model_name,
    *
  FROM ML.EVALUATE(MODEL `mgmt467project.netflix.churn_model`)
),
enhanced_eval AS (
  SELECT
    'Enhanced Model' AS model_name,
    *
  FROM ML.EVALUATE(MODEL `mgmt467project.netflix.churn_model_enhanced`)
)
SELECT *
FROM base_eval
UNION ALL
SELECT *
FROM enhanced_eval;


Query is running:   0%|          |

Downloading:   0%|          |

,model_name,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,Base Model,0.0,0.0,0.850892,0.0,0.421135,0.510065
1,Enhanced Model,0.0,0.0,0.860182,0.0,0.403663,0.539899


My Reflection: The enhanced model outperformed the base version, improving accuracy (0.8509 → 0.8601), ROC AUC (0.51 → 0.54), and reducing log loss (0.42 → 0.40). This indicates that engineered features—particularly watch-time behavior, binge patterns, and recency indicators—successfully captured richer user engagement signals, leading to more reliable churn predictions.